In [9]:
import pandas as pd
import numpy as np
import holidays
from prophet import Prophet
from typing import Dict, List, Tuple
import xgboost as xgb
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================

CONFIG = {
    'start_date': '2021-01-01',
    'outlier_percentile': 1.0,  # Remove top/bottom 1%
    'small_gap_hours': 6,
    'pollutants': ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25']
}

# ============================================================================
# FEATURE ENGINEERING
# ============================================================================

def add_temporal_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add comprehensive temporal features."""
    df = df.copy()
    df['datetime'] = pd.to_datetime(df['id'], format='%Y-%m-%d %H')
    
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['hour'] = df['datetime'].dt.hour
    df['dayofweek'] = df['datetime'].dt.dayofweek
    df['dayofyear'] = df['datetime'].dt.dayofyear
    df['week'] = df['datetime'].dt.isocalendar().week
    df['quarter'] = df['datetime'].dt.quarter
    
    # Cyclical encoding
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['dayofyear_sin'] = np.sin(2 * np.pi * df['dayofyear'] / 365)
    df['dayofyear_cos'] = np.cos(2 * np.pi * df['dayofyear'] / 365)
    
    # Binary flags
    df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
    df['is_rush_hour'] = ((df['hour'].between(7, 9)) | (df['hour'].between(17, 19))).astype(int)
    df['is_night'] = ((df['hour'] >= 22) | (df['hour'] <= 6)).astype(int)
    df['is_business_hours'] = ((df['hour'].between(9, 18)) & (df['dayofweek'] < 5)).astype(int)
    
    # Holidays
    fr_holidays = holidays.France(years=range(2020, 2026))
    df['is_holiday'] = df['datetime'].dt.date.isin(fr_holidays).astype(int)
    
    # Seasonal periods
    df['is_summer_vacation'] = df['month'].isin([7, 8]).astype(int)
    df['is_winter_vacation'] = (((df['month'] == 12) & (df['day'] >= 20)) | 
                                 ((df['month'] == 1) & (df['day'] <= 5))).astype(int)
    df['is_heating_season'] = ((df['month'] >= 10) | (df['month'] <= 4)).astype(int)
    
    return df


def add_covid_features(df: pd.DataFrame, covid_path: str) -> pd.DataFrame:
    """Add COVID stringency index."""
    df = df.copy()
    df['date'] = df['datetime'].dt.date
    
    try:
        covid_index = pd.read_csv(covid_path)
        covid_index['date'] = pd.to_datetime(covid_index['Date'], format='%Y%m%d').dt.date
        covid_index = (
            covid_index
            .query("CountryName == 'France'")
            [['date', 'StringencyIndex_Average']]
            .drop_duplicates()
        )
        df = df.merge(covid_index, on='date', how='left')
        df['StringencyIndex_Average'] = df['StringencyIndex_Average'].fillna(0)
    except:
        df['StringencyIndex_Average'] = 0
    
    df = df.drop(columns='date')
    return df


# ============================================================================
# DATA CLEANING
# ============================================================================

def filter_post_2021(df: pd.DataFrame, start_date: str) -> pd.DataFrame:
    """Filter data to start from specified date."""
    df = df.copy()
    df = df[df['datetime'] >= start_date].reset_index(drop=True)
    return df


def remove_outliers(df: pd.DataFrame, pollutants: List[str], percentile: float) -> pd.DataFrame:
    """Remove outliers beyond specified percentile."""
    df = df.copy()
    
    print(f"\nRemoving outliers (beyond {percentile}th and {100-percentile}th percentile):")
    for pollutant in pollutants:
        original_count = df[pollutant].notna().sum()
        
        lower = df[pollutant].quantile(percentile / 100)
        upper = df[pollutant].quantile(1 - percentile / 100)
        
        # Set outliers to NaN (will be imputed later)
        df.loc[(df[pollutant] < lower) | (df[pollutant] > upper), pollutant] = np.nan
        
        removed = original_count - df[pollutant].notna().sum()
        print(f"  {pollutant}: removed {removed} outliers (range: {lower:.2f} - {upper:.2f})")
    
    return df


# ============================================================================
# MISSING VALUE IMPUTATION
# ============================================================================

def identify_gaps(series: pd.Series) -> Tuple[List, List]:
    """Identify small (<6h) and large gaps."""
    is_missing = series.isna()
    
    # Find consecutive missing sequences
    gap_starts = []
    gap_lengths = []
    
    in_gap = False
    gap_start = None
    gap_length = 0
    
    for idx, missing in enumerate(is_missing):
        if missing and not in_gap:
            in_gap = True
            gap_start = idx
            gap_length = 1
        elif missing and in_gap:
            gap_length += 1
        elif not missing and in_gap:
            gap_starts.append(gap_start)
            gap_lengths.append(gap_length)
            in_gap = False
            gap_start = None
            gap_length = 0
    
    # Handle case where series ends with gap
    if in_gap:
        gap_starts.append(gap_start)
        gap_lengths.append(gap_length)
    
    small_gaps = [(s, l) for s, l in zip(gap_starts, gap_lengths) if l <= CONFIG['small_gap_hours']]
    large_gaps = [(s, l) for s, l in zip(gap_starts, gap_lengths) if l > CONFIG['small_gap_hours']]
    
    return small_gaps, large_gaps


def interpolate_small_gaps(df: pd.DataFrame, pollutants: List[str]) -> pd.DataFrame:
    """Interpolate small gaps (<6 hours)."""
    df = df.copy()
    df = df.sort_values('datetime').reset_index(drop=True)
    
    print("\nInterpolating small gaps (<6 hours):")
    for pollutant in pollutants:
        before_missing = df[pollutant].isna().sum()
        
        # Linear interpolation with limit
        df[pollutant] = df[pollutant].interpolate(
            method='linear', 
            limit=CONFIG['small_gap_hours'],
            limit_direction='both'
        )
        
        after_missing = df[pollutant].isna().sum()
        filled = before_missing - after_missing
        print(f"  {pollutant}: filled {filled} values")
    
    return df


def train_imputation_model(df: pd.DataFrame, pollutant: str, 
                          feature_cols: List[str]) -> xgb.XGBRegressor:
    """Train XGBoost model for imputing missing values."""
    # Get non-missing data
    train_data = df[df[pollutant].notna()].copy()
    
    if len(train_data) < 100:
        return None
    
    X = train_data[feature_cols]
    y = train_data[pollutant]
    
    model = xgb.XGBRegressor(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42,
        n_jobs=-1
    )
    
    model.fit(X, y)
    return model


def impute_large_gaps(df: pd.DataFrame, pollutants: List[str]) -> pd.DataFrame:
    """Use XGBoost to impute large gaps."""
    df = df.copy()
    
    # Features for imputation (temporal only, no other pollutants to avoid cascading errors)
    imputation_features = [
        'hour', 'dayofweek', 'month', 'dayofyear', 'quarter',
        'hour_sin', 'hour_cos', 'dayofweek_sin', 'dayofweek_cos',
        'month_sin', 'month_cos', 'dayofyear_sin', 'dayofyear_cos',
        'is_weekend', 'is_rush_hour', 'is_night', 'is_business_hours',
        'is_holiday', 'is_summer_vacation', 'is_winter_vacation', 'is_heating_season',
        'StringencyIndex_Average'
    ]
    
    print("\nImputing large gaps with XGBoost:")
    for pollutant in pollutants:
        missing_count = df[pollutant].isna().sum()
        
        if missing_count == 0:
            print(f"  {pollutant}: no missing values")
            continue
        
        # Train imputation model
        model = train_imputation_model(df, pollutant, imputation_features)
        
        if model is None:
            print(f"  {pollutant}: insufficient data, using forward fill")
            df[pollutant] = df[pollutant].ffill().bfill()
            continue
        
        # Predict missing values
        missing_mask = df[pollutant].isna()
        if missing_mask.sum() > 0:
            X_missing = df.loc[missing_mask, imputation_features]
            predictions = model.predict(X_missing)
            df.loc[missing_mask, pollutant] = np.maximum(0, predictions)
        
        print(f"  {pollutant}: imputed {missing_count} values")
    
    return df


# ============================================================================
# PROPHET MODELING
# ============================================================================

def train_prophet_model(df: pd.DataFrame, pollutant: str, 
                       regressors: List[str], seasonality_config: Dict) -> Prophet:
    """Train Prophet with strong seasonality."""
    prophet_df = pd.DataFrame({
        'ds': df['datetime'],
        'y': df[pollutant]
    })
    
    for regressor in regressors:
        prophet_df[regressor] = df[regressor].values
    
    prophet_df = prophet_df.dropna(subset=['y'])
    
    # Initialize Prophet with enhanced seasonality
    model = Prophet(
        yearly_seasonality=seasonality_config['yearly'],
        weekly_seasonality=seasonality_config['weekly'],
        daily_seasonality=seasonality_config['daily'],
        seasonality_mode=seasonality_config['mode'],
        changepoint_prior_scale=seasonality_config['changepoint_scale']
    )
    
    # Add custom seasonalities with more Fourier terms
    if seasonality_config.get('custom_yearly', False):
        model.add_seasonality(
            name='yearly_strong',
            period=365.25,
            fourier_order=seasonality_config['yearly_fourier']
        )
    
    if seasonality_config.get('custom_monthly', False):
        model.add_seasonality(
            name='monthly',
            period=30.5,
            fourier_order=seasonality_config['monthly_fourier']
        )
    
    # Add regressors
    for regressor in regressors:
        model.add_regressor(regressor)
    
    model.fit(prophet_df)
    return model


def train_all_models(df: pd.DataFrame, pollutants: List[str], 
                    regressors: List[str]) -> Dict[str, Prophet]:
    """Train Prophet models for all pollutants."""
    models = {}
    
    # Different seasonality configs per pollutant
    seasonality_configs = {
        'valeur_O3': {  # O3 has strongest seasonality
            'yearly': False,  # Use custom instead
            'weekly': True,
            'daily': True,
            'mode': 'multiplicative',
            'changepoint_scale': 0.05,
            'custom_yearly': True,
            'yearly_fourier': 20,  # Strong yearly pattern
            'custom_monthly': True,
            'monthly_fourier': 8
        },
        'default': {  # For other pollutants
            'yearly': False,
            'weekly': True,
            'daily': True,
            'mode': 'multiplicative',
            'changepoint_scale': 0.05,
            'custom_yearly': True,
            'yearly_fourier': 12,
            'custom_monthly': True,
            'monthly_fourier': 5
        }
    }
    
    print("\nTraining Prophet models with enhanced seasonality:")
    for pollutant in pollutants:
        config = seasonality_configs.get(pollutant, seasonality_configs['default'])
        model = train_prophet_model(df, pollutant, regressors, config)
        models[pollutant] = model
        print(f"  ✓ {pollutant}")
    
    return models


def generate_predictions(models: Dict[str, Prophet], test: pd.DataFrame,
                        pollutants: List[str], regressors: List[str]) -> pd.DataFrame:
    """Generate predictions for test set."""
    predictions = {}
    
    print("\nGenerating predictions:")
    for pollutant in pollutants:
        future_df = pd.DataFrame({'ds': test['datetime']})
        
        for regressor in regressors:
            future_df[regressor] = test[regressor].values
        
        forecast = models[pollutant].predict(future_df)
        predictions[pollutant] = np.maximum(0, forecast['yhat'].values)
        
        print(f"  ✓ {pollutant}: mean={predictions[pollutant].mean():.2f}, "
              f"median={np.median(predictions[pollutant]):.2f}")
    
    submission = test[['id']].copy()
    for pollutant in pollutants:
        submission[pollutant] = predictions[pollutant]
    
    return submission


# ============================================================================
# COMPARISON & REPORTING
# ============================================================================

def compare_with_friend(submission: pd.DataFrame, friend_path: str,
                       pollutants: List[str]) -> None:
    """Compare predictions with friend's submission."""
    friend = pd.read_csv(friend_path)
    
    print("\n" + "="*70)
    print("COMPARISON WITH FRIEND'S SUBMISSION")
    print("="*70)
    
    print(f"\n{'Pollutant':<15} {'Prophet':<12} {'Friend':<12} {'Diff':<10} {'% Diff'}")
    print("-" * 70)
    
    for p in pollutants:
        prophet_val = submission[p].mean()
        friend_val = friend[p].mean()
        diff = prophet_val - friend_val
        pct_diff = (diff / friend_val * 100) if friend_val != 0 else 0
        print(f"{p:<15} {prophet_val:<12.2f} {friend_val:<12.2f} {diff:<10.2f} {pct_diff:>6.1f}%")


def print_data_summary(train: pd.DataFrame, pollutants: List[str]) -> None:
    """Print summary of data quality."""
    print("\n" + "="*70)
    print("DATA SUMMARY")
    print("="*70)
    print(f"Training period: {train['datetime'].min()} to {train['datetime'].max()}")
    print(f"Total samples: {len(train)}")
    print(f"\nMissing values after imputation:")
    for p in pollutants:
        missing = train[p].isna().sum()
        print(f"  {p}: {missing} ({missing/len(train)*100:.2f}%)")


# ============================================================================
# MAIN PIPELINE
# ============================================================================

def main():
    print("="*70)
    print("PROPHET PIPELINE - ENHANCED SEASONALITY & SMART IMPUTATION")
    print("="*70)
    
    # Load data
    train = pd.read_csv('data/train.csv')
    test = pd.read_csv('data/test.csv')
    pollutants = CONFIG['pollutants']
    
    # Add features
    print("\nAdding features...")
    train = add_temporal_features(train)
    test = add_temporal_features(test)
    train = add_covid_features(train, 'data/OxCGRT_compact_national_v1.csv')
    test = add_covid_features(test, 'data/OxCGRT_compact_national_v1.csv')
    print("  ✓ Temporal & COVID features added")
    
    # Filter to 2021+
    train = filter_post_2021(train, CONFIG['start_date'])
    print(f"\n✓ Filtered to {CONFIG['start_date']} onwards: {len(train)} samples")
    
    # Remove outliers
    train = remove_outliers(train, pollutants, CONFIG['outlier_percentile'])
    
    # Impute missing values
    train = interpolate_small_gaps(train, pollutants)
    train = impute_large_gaps(train, pollutants)
    
    # Data summary
    print_data_summary(train, pollutants)
    
    # Define regressors
    regressors = [
        'hour', 'dayofweek', 'month', 'dayofyear', 'quarter',
        'hour_sin', 'hour_cos', 'dayofweek_sin', 'dayofweek_cos',
        'month_sin', 'month_cos', 'dayofyear_sin', 'dayofyear_cos',
        'is_weekend', 'is_rush_hour', 'is_night', 'is_business_hours',
        'is_holiday', 'is_summer_vacation', 'is_winter_vacation', 'is_heating_season',
        'StringencyIndex_Average'
    ]
    
    # Train models
    models = train_all_models(train, pollutants, regressors)
    
    # Generate predictions
    submission = generate_predictions(models, test, pollutants, regressors)
    
    # Save submission
    submission.to_csv('submission_prophet_enhanced.csv', index=False)
    print("\n✓ Saved: submission_prophet_enhanced.csv")
    
    # Compare with friend
    compare_with_friend(submission, 'prophet_new_predictions (5).csv', pollutants)
    
    print("\n" + "="*70)
    print("PIPELINE COMPLETE")
    print("="*70)
    print("\nKey improvements:")
    print("  • Started from 2021 (removed COVID anomaly)")
    print("  • Removed top/bottom 1% outliers")
    print("  • Smart imputation: interpolation + XGBoost")
    print("  • Enhanced seasonality (20 Fourier terms for O3)")
    print("  • Kept COVID feature for context")


if __name__ == "__main__":
    main()

PROPHET PIPELINE - ENHANCED SEASONALITY & SMART IMPUTATION

Adding features...
  ✓ Temporal & COVID features added

✓ Filtered to 2021-01-01 onwards: 32207 samples

Removing outliers (beyond 1.0th and 99.0th percentile):
  valeur_NO2: removed 581 outliers (range: 4.60 - 73.09)
  valeur_CO: removed 390 outliers (range: 0.10 - 0.60)
  valeur_O3: removed 639 outliers (range: 0.60 - 120.63)
  valeur_PM10: removed 516 outliers (range: 3.20 - 58.92)
  valeur_PM25: removed 623 outliers (range: 1.90 - 40.91)

Interpolating small gaps (<6 hours):
  valeur_NO2: filled 868 values
  valeur_CO: filled 595 values
  valeur_O3: filled 813 values
  valeur_PM10: filled 4882 values
  valeur_PM25: filled 916 values

Imputing large gaps with XGBoost:
  valeur_NO2: imputed 2213 values
  valeur_CO: imputed 11722 values
  valeur_O3: imputed 61 values
  valeur_PM10: imputed 1455 values
  valeur_PM25: imputed 126 values

DATA SUMMARY
Training period: 2021-01-01 00:00:00 to 2024-09-03 22:00:00
Total samples: 322

16:00:51 - cmdstanpy - INFO - Chain [1] start processing
16:01:05 - cmdstanpy - INFO - Chain [1] done processing


  ✓ valeur_NO2


16:01:06 - cmdstanpy - INFO - Chain [1] start processing
16:01:22 - cmdstanpy - INFO - Chain [1] done processing


  ✓ valeur_CO


...

  ✓ valeur_O3


16:01:47 - cmdstanpy - INFO - Chain [1] start processing
16:02:12 - cmdstanpy - INFO - Chain [1] done processing


  ✓ valeur_PM10


16:02:13 - cmdstanpy - INFO - Chain [1] start processing
16:02:33 - cmdstanpy - INFO - Chain [1] done processing


  ✓ valeur_PM25

Generating predictions:
  ✓ valeur_NO2: mean=16.65, median=16.68
  ✓ valeur_CO: mean=0.17, median=0.17
  ✓ valeur_O3: mean=53.66, median=52.50
  ✓ valeur_PM10: mean=12.91, median=13.07
  ✓ valeur_PM25: mean=6.08, median=6.13

✓ Saved: submission_prophet_enhanced.csv

COMPARISON WITH FRIEND'S SUBMISSION

Pollutant       Prophet      Friend       Diff       % Diff
----------------------------------------------------------------------
valeur_NO2      16.65        20.19        -3.54       -17.5%
valeur_CO       0.17         0.18         -0.01        -3.3%
valeur_O3       53.66        42.71        10.95        25.6%
valeur_PM10     12.91        14.16        -1.25        -8.8%
valeur_PM25     6.08         8.88         -2.80       -31.5%

PIPELINE COMPLETE

Key improvements:
  • Started from 2021 (removed COVID anomaly)
  • Removed top/bottom 1% outliers
  • Smart imputation: interpolation + XGBoost
  • Enhanced seasonality (20 Fourier terms for O3)
  • Kept COVID feature for

In [10]:
import pandas as pd

# Load both submissions
my_submission = pd.read_csv('submission_prophet_enhanced.csv')
friend_submission = pd.read_csv('prophet_new_predictions (5).csv')

# Replace O3 with friend's O3
my_submission['valeur_O3'] = friend_submission['valeur_O3']

# Save hybrid submission
my_submission.to_csv('submission_prophet_hybrid_o3.csv', index=False)

print("✓ Created hybrid submission with friend's O3")
print("\nPollutant sources:")
print("  • valeur_NO2:  Prophet (yours)")
print("  • valeur_CO:   Prophet (yours)")
print("  • valeur_O3:   Friend's model ✓")
print("  • valeur_PM10: Prophet (yours)")
print("  • valeur_PM25: Prophet (yours)")

✓ Created hybrid submission with friend's O3

Pollutant sources:
  • valeur_NO2:  Prophet (yours)
  • valeur_CO:   Prophet (yours)
  • valeur_O3:   Friend's model ✓
  • valeur_PM10: Prophet (yours)
  • valeur_PM25: Prophet (yours)


In [ ]:
import pandas as pd
import numpy as np
import holidays
from prophet import Prophet
from typing import Dict, List
from lightgbm import LGBMRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import warnings
warnings.filterwarnings('ignore')

# Import the utility functions (assuming they're in src/features/feature_engineering_maxim.py)
# If not, copy them to the top of this file

# ============================================================================
# CONFIGURATION
# ============================================================================

CONFIG = {
    'start_date': '2021-01-01',
    'outlier_percentile': 1.0,
    'pollutants': ['valeur_NO2', 'valeur_CO', 'valeur_O3', 'valeur_PM10', 'valeur_PM25'],
    'weather_file': 'data/paris_meteostat_2020_2025.csv'
}

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def cyclical_encoding(col):
    """Create sin/cos encoding for cyclical features."""
    if not isinstance(col, pd.Series):
        raise TypeError("Input must be a pandas Series")
    
    max_val = col.max()
    scaled = (col * 2 * np.pi) / max_val
    return np.sin(scaled), np.cos(scaled)


def get_features(data: pd.DataFrame, idcol: str = "id", encode_cyclical: bool = True):
    """Add temporal features with cyclical encoding."""
    fr_holidays = holidays.France()
    df = data.copy()
    
    df[idcol] = pd.to_datetime(df[idcol], format='%Y-%m-%d %H')
    df["year"] = df[idcol].dt.year
    df["month"] = df[idcol].dt.month
    df["hour"] = df[idcol].dt.hour
    df["weekday"] = df[idcol].dt.dayofweek
    df["is_weekend"] = (df[idcol].dt.dayofweek >= 5).astype(int)
    df["is_holiday"] = df[idcol].dt.date.apply(lambda x: x in fr_holidays).astype(int)
    
    if encode_cyclical:
        cyclical_features = ["month", "weekday", "hour"]
        for col in cyclical_features:
            df[f"{col}_sin"], df[f"{col}_cos"] = cyclical_encoding(df[col])
        df.drop(columns=cyclical_features, inplace=True)
    
    return df.sort_values(by=idcol, ascending=True)


def fill_weather_gaps(df_weather: pd.DataFrame, start=None, end=None) -> pd.DataFrame:
    """Fill missing rows and interpolate weather data."""
    df = df_weather.copy()
    
    df["time"] = pd.to_datetime(df["time"], errors="coerce")
    df = df.dropna(subset=["time"]).sort_values("time")
    df = df[~df["time"].duplicated(keep="first")]
    
    if start is None:
        start = df["time"].min().floor("H")
    else:
        start = pd.to_datetime(start).floor("H")
    if end is None:
        end = df["time"].max().ceil("H")
    else:
        end = pd.to_datetime(end).ceil("H")
    
    full_idx = pd.date_range(start, end, freq="H")
    df = df.set_index("time").reindex(full_idx)
    
    cont_cols = ["temp", "dwpt", "rhum", "wdir", "wspd", "pres"]
    for col in cont_cols:
        if col in df.columns:
            df[col] = df[col].interpolate(method="time", limit_direction="both")
    
    if "prcp" in df.columns:
        df["prcp"] = df["prcp"].fillna(0)
    
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.ffill(limit=3).bfill(limit=3)
    df = df.reset_index().rename(columns={"index": "time"})
    
    cols_order = [c for c in ["time","temp","dwpt","rhum","prcp","wdir","wspd","pres"] if c in df.columns]
    df = df[cols_order + [c for c in df.columns if c not in cols_order]]
    
    return df


def enrich_weather_features(df_weather: pd.DataFrame, tz_local: str = "Europe/Paris") -> pd.DataFrame:
    """Enrich weather data with physical features."""
    df = df_weather.copy()
    df = df.drop_duplicates(subset=["time"]).sort_values("time").reset_index(drop=True)
    df["time"] = pd.to_datetime(df["time"], errors="coerce")
    
    # Wind features
    wdir = df["wdir"].fillna(0)
    wdir_rad = np.deg2rad(wdir % 360)
    df["wind_u"] = -df["wspd"] * np.sin(wdir_rad)
    df["wind_v"] = -df["wspd"] * np.cos(wdir_rad)
    
    # Humidity physics
    es = 0.6108 * np.exp(17.27 * df["temp"] / (df["temp"] + 237.3))
    ea = 0.6108 * np.exp(17.27 * df["dwpt"] / (df["dwpt"] + 237.3))
    df["vpd"] = (es - ea).clip(lower=0)
    df["abs_humidity"] = 216.7 * (ea / (df["temp"] + 273.15))
    
    # Rain features
    df["is_wet_hour"] = (df["prcp"].fillna(0) > 0).astype(int)
    df["rain_3h_sum"] = df["prcp"].rolling(3, min_periods=1).sum()
    
    # Pressure anomaly
    df["pres_anom_7d"] = df["pres"] - df["pres"].rolling(24 * 7, min_periods=12).mean()
    
    # Solar geometry
    lat = np.deg2rad(48.8566)
    df_local = df.copy()
    df_local["time_tz"] = pd.to_datetime(df_local["time"]).dt.tz_localize(
        tz_local, ambiguous="NaT", nonexistent="shift_forward"
    )
    doy = df_local["time_tz"].dt.dayofyear.values
    hour = df_local["time_tz"].dt.hour.values + df_local["time_tz"].dt.minute.values / 60.0
    
    decl = 23.44 * np.pi / 180 * np.sin(2 * np.pi * (284 + doy) / 365)
    h_angle = np.pi / 12 * (hour - 12)
    solar_elev = np.arcsin(np.sin(lat)*np.sin(decl) + np.cos(lat)*np.cos(decl)*np.cos(h_angle))
    df["solar_elev_sin"] = np.sin(np.clip(solar_elev, 0, np.pi/2))
    df["is_daylight"] = (solar_elev > 0).astype(int)
    
    df = df.replace([np.inf, -np.inf], np.nan).ffill(limit=3)
    
    keep_cols = [
        "time", "temp", "rhum", "pres",
        "wind_u", "wind_v",
        "prcp", "is_wet_hour", "rain_3h_sum",
        "vpd", "abs_humidity",
        "solar_elev_sin", "is_daylight"
    ]
    
    return df[keep_cols]


def merge_weather(main: pd.DataFrame, weather: pd.DataFrame, 
                 main_col: str = "id", weather_col: str = "time") -> pd.DataFrame:
    """Merge weather data with main dataframe."""
    assert main[main_col].duplicated().sum() == 0, "Non-unique dates in main"
    assert weather[weather_col].duplicated().sum() == 0, "Non-unique dates in weather"
    
    output = main.merge(weather, left_on=main_col, right_on=weather_col, how="left")
    return output


def make_lagged_features(df: pd.DataFrame,
                        pollutants: list,
                        lags=(1, 3, 6, 12, 24),
                        rolls=(3, 6, 12, 24)) -> pd.DataFrame:
    """Create lagged and rolling features for pollutants."""
    df = df.copy().sort_values("id")
    
    for col in pollutants:
        if col not in df.columns:
            continue
        
        for lag in lags:
            df[f"{col}_lag{lag}"] = df[col].shift(lag)
        
        for win in rolls:
            roll = df[col].rolling(window=win, min_periods=1)
            df[f"{col}_roll{win}_mean"] = roll.mean()
            df[f"{col}_roll{win}_std"] = roll.std()
    
    df = df.loc[:, ~df.columns.duplicated()]
    return df


def impute_pollutants(df: pd.DataFrame, pollutants: list, 
                     weather_cols: list, cal_cols: list) -> pd.DataFrame:
    """Impute missing pollutant values using IterativeImputer with LightGBM."""
    df = df.copy()
    
    for p in pollutants:
        df[p + "_missing"] = df[p].isna().astype(int)
    
    lag_roll_cols = []
    for p in pollutants:
        lag_roll_cols += [c for c in df.columns if c.startswith(p + "_lag")]
        lag_roll_cols += [c for c in df.columns if c.startswith(p + "_roll")]
    
    cols = pollutants + weather_cols + cal_cols + lag_roll_cols + [p+"_missing" for p in pollutants]
    cols = [c for c in cols if c in df.columns]
    
    X = df[cols].astype(float)
    
    print(f"Imputation using {len(cols)} features")
    
    imp = IterativeImputer(
        estimator=LGBMRegressor(n_estimators=50, max_depth=5, random_state=42, verbose=-1),
        max_iter=10,
        sample_posterior=False,
        initial_strategy="median",
        skip_complete=True,
        random_state=42,
        verbose=0
    )
    
    X_imp = imp.fit_transform(X)
    X_imp_df = pd.DataFrame(X_imp, columns=X.columns, index=X.index)
    
    for p in pollutants:
        mask = df[p].isna()
        df.loc[mask, p] = X_imp_df.loc[mask, p]
        df[p] = df[p].clip(lower=0)
    
    # Drop missing indicator columns
    df = df.drop(columns=[p+"_missing" for p in pollutants], errors='ignore')
    
    return df


# ============================================================================
# DATA CLEANING
# ============================================================================

def filter_post_2021(df: pd.DataFrame, start_date: str) -> pd.DataFrame:
    """Filter data starting from specified date."""
    df = df.copy()
    df = df[df['id'] >= start_date].reset_index(drop=True)
    return df


def remove_outliers(df: pd.DataFrame, pollutants: List[str], percentile: float) -> pd.DataFrame:
    """Remove outliers beyond specified percentile."""
    df = df.copy()
    
    print(f"\nRemoving outliers (beyond {percentile}th and {100-percentile}th percentile):")
    for pollutant in pollutants:
        if pollutant not in df.columns:
            continue
            
        original_count = df[pollutant].notna().sum()
        
        lower = df[pollutant].quantile(percentile / 100)
        upper = df[pollutant].quantile(1 - percentile / 100)
        
        df.loc[(df[pollutant] < lower) | (df[pollutant] > upper), pollutant] = np.nan
        
        removed = original_count - df[pollutant].notna().sum()
        print(f"  {pollutant}: removed {removed} outliers")
    
    return df


# ============================================================================
# PROPHET MODELING
# ============================================================================

def train_prophet_model(df: pd.DataFrame, pollutant: str, 
                       regressors: List[str], enhanced_o3: bool = False) -> Prophet:
    """Train Prophet with enhanced seasonality."""
    prophet_df = pd.DataFrame({
        'ds': df['id'],
        'y': df[pollutant]
    })
    
    for regressor in regressors:
        if regressor in df.columns:
            prophet_df[regressor] = df[regressor].values
    
    prophet_df = prophet_df.dropna(subset=['y'])
    
    # Enhanced seasonality for O3
    if enhanced_o3 and pollutant == 'valeur_O3':
        model = Prophet(
            yearly_seasonality=False,
            weekly_seasonality=True,
            daily_seasonality=True,
            seasonality_mode='multiplicative',
            changepoint_prior_scale=0.05
        )
        model.add_seasonality(name='yearly_strong', period=365.25, fourier_order=20)
        model.add_seasonality(name='monthly', period=30.5, fourier_order=8)
    else:
        model = Prophet(
            yearly_seasonality=False,
            weekly_seasonality=True,
            daily_seasonality=True,
            seasonality_mode='multiplicative',
            changepoint_prior_scale=0.05
        )
        model.add_seasonality(name='yearly', period=365.25, fourier_order=12)
        model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    
    for regressor in regressors:
        if regressor in prophet_df.columns:
            model.add_regressor(regressor)
    
    model.fit(prophet_df)
    return model


def train_all_models(df: pd.DataFrame, pollutants: List[str], 
                    regressors: List[str]) -> Dict[str, Prophet]:
    """Train Prophet models for all pollutants."""
    models = {}
    
    print("\nTraining Prophet models:")
    for pollutant in pollutants:
        enhanced = (pollutant == 'valeur_O3')
        model = train_prophet_model(df, pollutant, regressors, enhanced_o3=enhanced)
        models[pollutant] = model
        print(f"  ✓ {pollutant}" + (" (enhanced seasonality)" if enhanced else ""))
    
    return models


def generate_predictions(models: Dict[str, Prophet], test: pd.DataFrame,
                        pollutants: List[str], regressors: List[str]) -> pd.DataFrame:
    """Generate predictions for test set."""
    predictions = {}
    
    print("\nGenerating predictions:")
    for pollutant in pollutants:
        future_df = pd.DataFrame({'ds': test['id']})
        
        for regressor in regressors:
            if regressor in test.columns:
                future_df[regressor] = test[regressor].values
        
        forecast = models[pollutant].predict(future_df)
        predictions[pollutant] = np.maximum(0, forecast['yhat'].values)
        
        print(f"  ✓ {pollutant}: mean={predictions[pollutant].mean():.2f}")
    
    submission = test[['id']].copy()
    for pollutant in pollutants:
        submission[pollutant] = predictions[pollutant]
    
    return submission


def compare_with_friend(submission: pd.DataFrame, friend_path: str,
                       pollutants: List[str]) -> None:
    """Compare predictions with friend's submission."""
    friend = pd.read_csv(friend_path)
    
    print("\n" + "="*70)
    print("COMPARISON WITH FRIEND'S SUBMISSION")
    print("="*70)
    
    print(f"\n{'Pollutant':<15} {'Prophet':<12} {'Friend':<12} {'Diff':<10} {'% Diff'}")
    print("-" * 70)
    
    for p in pollutants:
        prophet_val = submission[p].mean()
        friend_val = friend[p].mean()
        diff = prophet_val - friend_val
        pct_diff = (diff / friend_val * 100) if friend_val != 0 else 0
        print(f"{p:<15} {prophet_val:<12.2f} {friend_val:<12.2f} {diff:<10.2f} {pct_diff:>6.1f}%")


# ============================================================================
# MAIN PIPELINE
# ============================================================================

def main():
    print("="*70)
    print("PROPHET PIPELINE - USING MAXIM'S FEATURE ENGINEERING")
    print("="*70)
    
    pollutants = CONFIG['pollutants']
    
    # Load data
    train = pd.read_csv('data/train.csv')
    test = pd.read_csv('data/test.csv')
    
    print(f"\nOriginal data: Train={len(train)}, Test={len(test)}")
    
    # Add temporal features
    print("\nAdding temporal features...")
    train = get_features(train, idcol='id', encode_cyclical=True)
    test = get_features(test, idcol='id', encode_cyclical=True)
    print("  ✓ Temporal features with cyclical encoding")
    
    # Load and process weather
    print("\nProcessing weather data...")
    weather_raw = pd.read_csv(CONFIG['weather_file'])
    weather_raw = weather_raw.rename(columns={'Unnamed: 0': 'time'})
    
    # Fill gaps and enrich
    weather = fill_weather_gaps(weather_raw)
    weather = enrich_weather_features(weather)
    print("  ✓ Weather data processed and enriched")
    
    # Merge weather (only for training - test won't use weather for Prophet regressors)
    train = merge_weather(train, weather, main_col='id', weather_col='time')
    print("  ✓ Weather merged to training data")
    
    # Filter to 2021+
    train = filter_post_2021(train, CONFIG['start_date'])
    print(f"\n✓ Filtered to {CONFIG['start_date']} onwards: {len(train)} samples")
    
    # Remove outliers
    train = remove_outliers(train, pollutants, CONFIG['outlier_percentile'])
    
    # Create lagged features
    print("\nCreating lagged features...")
    train = make_lagged_features(train, pollutants, lags=(1, 3, 6, 12, 24), rolls=(3, 6, 12, 24))
    print("  ✓ Lagged and rolling features created")
    
    # Impute missing values
    print("\nImputing missing values with IterativeImputer + LightGBM...")
    weather_cols = ['temp', 'rhum', 'pres', 'wind_u', 'wind_v', 'prcp', 
                   'vpd', 'abs_humidity', 'solar_elev_sin']
    cal_cols = ['month_sin', 'month_cos', 'weekday_sin', 'weekday_cos', 
               'hour_sin', 'hour_cos', 'is_weekend', 'is_holiday']
    
    train = impute_pollutants(train, pollutants, weather_cols, cal_cols)
    
    print("\nMissing values after imputation:")
    for p in pollutants:
        missing = train[p].isna().sum()
        print(f"  {p}: {missing}")
    
    # Define regressors (only temporal - no weather/lags for Prophet)
    regressors = [
        'month_sin', 'month_cos', 'weekday_sin', 'weekday_cos',
        'hour_sin', 'hour_cos', 'is_weekend', 'is_holiday', 'year'
    ]
    
    print(f"\nUsing {len(regressors)} regressors for Prophet")
    
    # Train models
    models = train_all_models(train, pollutants, regressors)
    
    # Generate predictions
    submission = generate_predictions(models, test, pollutants, regressors)
    
    # Save submission
    submission.to_csv('submission_prophet_maxim.csv', index=False)
    print("\n✓ Saved: submission_prophet_maxim.csv")
    
    # Compare with friend
    compare_with_friend(submission, 'prophet_new_predictions (5).csv', pollutants)
    
    print("\n" + "="*70)
    print("PIPELINE COMPLETE")
    print("="*70)
    print("\nKey features:")
    print("  • Maxim's feature engineering functions")
    print("  • Weather from Meteostat with physical features")
    print("  • IterativeImputer with LightGBM")
    print("  • Enhanced seasonality (20 Fourier for O3)")
    print("  • Started from 2021, removed outliers")


if __name__ == "__main__":
    main()

PROPHET PIPELINE - USING MAXIM'S FEATURE ENGINEERING

Original data: Train=40991, Test=504

Adding temporal features...
  ✓ Temporal features with cyclical encoding

Processing weather data...
  ✓ Weather data processed and enriched
  ✓ Weather merged to training data

✓ Filtered to 2021-01-01 onwards: 32207 samples

Removing outliers (beyond 1.0th and 99.0th percentile):
  valeur_NO2: removed 581 outliers
  valeur_CO: removed 390 outliers
  valeur_O3: removed 639 outliers
  valeur_PM10: removed 516 outliers
  valeur_PM25: removed 623 outliers

Creating lagged features...
  ✓ Lagged and rolling features created

Imputing missing values with IterativeImputer + LightGBM...
Imputation using 92 features
